### Pre-Requisite: create environment install requirements-mgmt.txt

Create either a conda or python virtual environment and install the management packages:
```
pip install -r requirements-mgmt.txt
```

**Do NOT** install these packages into a global python interpreter, always use a conda or virtual environment or you may need to re-install python at some point.

In [13]:
# config variables
from azure.identity import DefaultAzureCredential

credential=DefaultAzureCredential()
resource_group_name = "dantaylo_sdktest_rg3"
hub_name = "dantaylo_sdktest_hub3"
project_name = "dantaylo-sdktest-project3"
location = "eastus"
subscription_id = "597966d1-829f-417e-9950-8189061ec09c"
aiservices_resource_name = "dantaylo-sdktest-aiservices4"
aisearch_resource_name = "dantaylo-sdktest-aisearch3"
aisearch_index_name = "dantaylo_aisearch_index"
azure_openai_chat_model = "gpt-35-turbo"
azure_openai_embedding_model = "text-embedding-ada-002"

In [4]:
# Create a resource group
from azure.mgmt.resource import ResourceManagementClient
resource_client = ResourceManagementClient(credential, subscription_id)

resource_group = resource_client.resource_groups.create_or_update(
    resource_group_name, {"location": location}
)

In [5]:
# Create an ML client
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
client = MLClient(workspace_name="", 
    resource_group_name=resource_group_name, 
    subscription_id=subscription_id, 
    credential=credential)

# Create a Hub
from azure.ai.ml.entities import Hub
hub_definition = Hub(name=hub_name, location=location)
print("Creating Hub, this will take a couple minutes...")
created_hub = client.workspaces.begin_create(workspace=hub_definition).result()

# Create a Project
from azure.ai.ml.entities import Project
project_definition = Project(name=project_name, hub_id=created_hub.id, location=location)
created_project = client.workspaces.begin_create(workspace=project_definition).result()

Class Hub: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Creating Hub, this will take a couple minutes...


The deployment request dantaylo_sdktest_hub3-3346709 was accepted. ARM deployment URI for reference: 
https://portal.azure.com//#blade/HubsExtension/DeploymentDetailsBlade/overview/id/%2Fsubscriptions%2F597966d1-829f-417e-9950-8189061ec09c%2FresourceGroups%2Fdantaylo_sdktest_rg3%2Fproviders%2FMicrosoft.Resources%2Fdeployments%2Fdantaylo_sdktest_hub3-3346709
Creating Key Vault: (dantaylokeyvault4b831185  ) ..  Done (24s)
Creating Storage Account: (dantaylostoragef66d8f3cd  )  Done (25s)
Creating AzureML Workspace: (dantaylo_sdktest_hub3  ) ..  Done (17s)
Total time : 44s

The deployment request dantaylo-sdktest-project3-8832763 was accepted. ARM deployment URI for reference: 
https://portal.azure.com//#blade/HubsExtension/DeploymentDetailsBlade/overview/id/%2Fsubscriptions%2F597966d1-829f-417e-9950-8189061ec09c%2FresourceGroups%2Fdantaylo_sdktest_rg3%2Fproviders%2FMicrosoft.Resources%2Fdeployments%2Fdantaylo-sdktest-project3-8832763
Creating AzureML Workspace: (dantaylo-sdktest-project3

In [14]:
# Create AI Services resource, this may take a few minutes

from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.ai.ml import MLClient
from azure.ai.ml.entities import AzureAIServicesConnection

aiservices_client = CognitiveServicesManagementClient(
    credential=DefaultAzureCredential(), subscription_id=subscription_id
)

aiservices_account = aiservices_client.accounts.begin_create(
    resource_group_name=resource_group_name,
    account_name=aiservices_resource_name,
    account={
        "sku": {"name": "S0"},
        "kind": "AIServices",
        "location": location,
    },
).result()


HttpResponseError: (BadRequest) Properties is required for this resouce.
Code: BadRequest
Message: Properties is required for this resouce.

In [ ]:
# Create a connection to the hub, this will create two connections:
#  - An Azure OpenAI Connection (for using the Azure OpenAI service and models)
#  - An AI Services Connection (for using all other AI Services)

aiservices_key = aiservices_client.accounts.list_keys(
    resource_group_name=resource_group_name,
    account_name=aiservices_resource_name,
).key1

aiservices_endpoint = aiservices_client.accounts.get(
    resource_group_name=resource_group_name,
    account_name=aiservices_resource_name
).properties.endpoint

resource_id=f"/subscriptions/{subscription_id}/resourceGroups/{resource_group_name}/providers/Microsoft.CognitiveServices/accounts/{aiservices_resource_name}"
aiservices_connection = AzureAIServicesConnection(
    name="azure_aiservices_connection",
    api_key=aiservices_key,
    target=aiservices_endpoint,
    ai_services_resource_id=resource_id)

ml_client = MLClient(workspace_name=hub_name, 
    resource_group_name=resource_group_name, 
    subscription_id=subscription_id, 
    credential=credential)

result = ml_client.connections.create_or_update(connection=aiservices_connection)

In [ ]:
# Create model deployments
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient

client = CognitiveServicesManagementClient(
    credential=credential,
    subscription_id=subscription_id,
)

deployment = client.deployments.begin_create_or_update(
    resource_group_name=resource_group_name,
    deployment_name=azure_openai_chat_model,
    account_name=aiservices_resource_name,
    deployment={
        "properties": {
            "model": {
                "format": "OpenAI",
                "name": azure_openai_chat_model,
                "version": "0301",
            }
        },
        "sku": {"capacity": 50, "name": "Standard"},
    },
).result()

deployment = client.deployments.begin_create_or_update(
    resource_group_name=resource_group_name,
    deployment_name=azure_openai_embedding_model,
    account_name=aiservices_resource_name,
    deployment={
        "properties": {
            "model": {
                "format": "OpenAI",
                "name": azure_openai_embedding_model,
                "version": "2",
            }
        },
        "sku": {"capacity": 50, "name": "Standard"},
    },
).result()

In [ ]:
# Create AIServices Resource
from azure.mgmt.search import SearchManagementClient
from azure.ai.ml import MLClient, ApiKeyConfiguration, AzureAISearchConnection

# Create AI Search resource
search_client = SearchManagementClient(
    credential=credential, subscription_id=subscription_id
)

search = search_client.services.begin_create_or_update(
    resource_group_name=resource_group_name,
    search_service_name=aisearch_resource_name,
    service={
        "location": location,
        "sku": {"name": "standard"},
        # "properties": {"hostingMode": "default", "partitionCount": 1, "replicaCount": 3},
    },
).result()

# create connection to the hub
aisearch_key = search_client.admin_keys.get(resource_group_name, aisearch_resource_name).primary_key
aisearch_endpoint = f"https://{aisearch_resource_name}.search.windows.net"

aisearch_connection = AzureAISearchConnection(
    name="azure_aisearch_connection",
    endpoint=aisearch_endpoint,
    credentials=ApiKeyConfiguration(key=aisearch_key),
)

# Create an ML client
ml_client = MLClient(workspace_name=hub_name, 
    resource_group_name=resource_group_name, 
    subscription_id=subscription_id, 
    credential=credential)

ml_client.connections.create_or_update(aisearch_connection)